# SQL  проект сервиса для чтения книг

Цель проекта - проанализировать базу данных сервиса для чтения книг с помощью создания SQL запросов.

Задачи проекта:

- посчитать, сколько книг вышло после 1 января 2000 года
- для каждой книги рассчитать количество обзоров и среднюю оценку
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц
- определить автора с самой высокой средней оценкой книг
- рассчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

#### Содержание проекта:

1. [Изучение общей информации.](#Шаг-1.-Изучение-общей-информации.)
2. Выясним, сколько книг вышло после 1 января 2000 года.
3. Подсчитаем для каждой книги количество обзоров и среднюю оценку.
4. Определим издательство, выпускающее больше всего книг толще 50 страниц.
5. Определим автора с самой высокой средней оценкой книг.
6. Подсчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
7. [Общий вывод](#Шаг-7.-Общий-вывод.)

### Шаг 1. Изучение общей информации. 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                     db_config['pwd'],
                     db_config['host'],
                     db_config['port'],
                     db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Чтение данных и визуалиция таблиц

In [3]:
pd.io.sql.read_sql('SELECT * FROM books LIMIT 5', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` содержит данные о книгах, имеет 6 колонок:
- `book_id` — идентификатор книги
- `author_id` — идентификатор автора
- `title` — название книги
- `num_pages` — количество страниц
- `publication_date` — дата публикации книги
- `publisher_id` — идентификатор издателя.
 

In [4]:
pd.io.sql.read_sql('SELECT * FROM authors LIMIT 5', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` содержит данные об авторах, имеет 2 колонки:

- `author_id` — идентификатор автора
- `author` — имя автора.

In [5]:
pd.io.sql.read_sql('SELECT * FROM publishers LIMIT 5', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` содержит данные об издательствах, имеет 2 колонки:

- `publisher_id` — идентификатор издательства
- `publisher` — название издательства.

In [6]:
pd.io.sql.read_sql('SELECT * FROM ratings LIMIT 5', con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `reviews` содержит данные о пользовательских оценках книг, имеет 4 колонки:

- `rating_id` — идентификатор оценки
- `book_id` — идентификатор книги
- `username` - имя пользователя, оставившего оценку
- `rating` - оценка книги.                             

In [7]:
pd.io.sql.read_sql('SELECT * FROM reviews LIMIT 5', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` содержит данные о пользовательских обзорах на книги, имеет 4 колонки:

- `review_id` — идентификатор обзора
- `book_id` — идентификатор книги
- `username` - имя пользователя, написавшего обзор
- `text` - текст обзора.

#### Вывод 
Изучили данные для анализа, представленные в 5 таблицах.

### Шаг 2. Выясним, сколько книг вышло после 1 января 2000 года.

In [8]:
query = '''
SELECT
    COUNT(book_id) as book_cnt
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''
a = pd.io.sql.read_sql(query, con = engine)
print('Количество опубликованных книг после 1 января 2020  -', a['book_cnt'][0])

Количество опубликованных книг после 1 января 2020  - 819


### Шаг 3. Подсчитаем для каждой книги количество обзоров и среднюю оценку.

In [9]:
query = '''
SELECT
    books.book_id,
    books.title,
    COUNT(DISTINCT review_id) as review_cnt,
    AVG(rating) as avg_rating
FROM 
    books
INNER JOIN reviews ON  reviews.book_id = books.book_id
INNER JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    review_cnt DESC
'''
print('Количество обзоров и средний рейтинг для каждой книги')
pd.io.sql.read_sql(query, con = engine)

Количество обзоров и средний рейтинг для каждой книги


,book_id,title,review_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,854,The Road,6,3.772727
2,656,The Book Thief,6,4.264151
3,734,The Glass Castle,6,4.206897
4,963,Water for Elephants,6,3.977273
...,...,...,...,...
989,465,Naked Empire (Sword of Truth #8),1,3.500000
990,446,Moo Baa La La La!,1,3.000000
991,431,Merrick (The Vampire Chronicles #7),1,4.000000
992,92,Babyville,1,3.500000


Больше всего пользовательских обзоров 7 штук было написано для книги `Twilight`- Сумерки. 

### Шаг 4. Определим издательство, выпускающее больше всего книг толще 50 страниц.

In [10]:
query = '''
SELECT
    publisher,
    COUNT(book_id) as book_cnt
FROM
    publishers
LEFT JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
LIMIT 3
'''
d = pd.io.sql.read_sql(query, con = engine)
display(d)
print('Издательство, выпускающее больше всего книг толщиной более 50 страниц - это ', d['publisher'][0], ', число книг - ', d['book_cnt'][0])

,publisher,book_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


Издательство, выпускающее больше всего книг толщиной более 50 страниц - это  Penguin Books , число книг -  42


### Шаг 5. Определим автора с самой высокой средней оценкой книг.

In [11]:
query = '''
SELECT
    author,
    AVG(sub.avg_rating) as total_avg_rating
FROM
    (SELECT
        author,
        books.book_id,
        AVG(rating) as avg_rating
    FROM 
        authors
    LEFT JOIN books ON books.author_id = authors.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        books.book_id
    HAVING
        COUNT(rating) > 50) as sub
GROUP BY
    author
ORDER BY
    total_avg_rating DESC
LIMIT 3
'''
f = pd.io.sql.read_sql(query, con = engine)
display(f)
print('Автор с самой высокой средней оценкой', f['total_avg_rating'][0], 'балла - это ', f['author'][0])

,author,total_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446


Автор с самой высокой средней оценкой 4.283844038868471 балла - это  J.K. Rowling/Mary GrandPré


### Шаг 6. Подсчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''
SELECT
    AVG(sub_text_cnt.text_cnt) AS avg_text_cnt
FROM
    (SELECT
        COUNT(text) as text_cnt
    FROM 
        reviews
    WHERE username IN
        (SELECT username
        FROM 
            ratings
        GROUP BY
            username
        HAVING
            COUNT(ratings) > 50)
    GROUP BY 
            username
    ) AS sub_text_cnt  
'''
b = pd.io.sql.read_sql(query, con = engine)
display(b)
print('В среднем пользователь, который поставил больше 50 оценок, пишет {:.2f}'.format(b['avg_text_cnt'][0]),'обзора')

,avg_text_cnt
0,24.333333


В среднем пользователь, который поставил больше 50 оценок, пишет 24.33 обзора


### Шаг 7. Общий вывод.

По итогам запросов для данных сервиса для чтения книг, можно сделать следующие выводы:

- после 1 января 2000 года было выпущенно 819 книг
- книга Сумерки насчитывает больше всего пользовательских обзоров (7) и имеет среднюю оценку в 3.7 балла
- издательство Penguin Books выпустило больше всех книг (42)
- автор с самой высокой средней оценкой книг - Джоан Роулинг (рейтинг 4,3)
- в среднем, пользователь который поставил больше 50 оценок, пишет 24,3 текстовых обзора.